In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import os
import json
import pandas as pd

In [ ]:
! pip3 install scikit-surprise==1.0.5
! pip3 install -U scikit-learn

In [ ]:

from surprise import Dataset
from surprise import Reader
from surprise import SVD, evaluate, accuracy
from surprise import dump
from sklearn.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

# Testing data


In [ ]:
#opening data

file_path = os.path.expanduser('/content/drive/MyDrive/Project/SEM 5/Yelp Dataset/FilteredData/finaldata.csv')

In [ ]:
#format for reading data
reader = Reader(line_format='user item rating', sep=',')


In [ ]:
#loading the data
data = Dataset.load_from_file(file_path, reader=reader)

In [ ]:
#splitting data into 5 parts
data.split(n_folds=5)

# SVD Model

In [ ]:
#single value decomposition model used for dimensionality reduction using matrix factorization
svd = SVD()

In [ ]:
#MAE - Mean absolute error (difference between the prediction of an observation and the true value of that observation.) (Lower is better accuracy)
#RMSE - root mean square error (the measure of how well a regression line fits the data points) (Lower is better)
evaluate(svd, data, measures=['RMSE', 'MAE'])

/usr/local/lib/python3.7/dist-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/usr/local/lib/python3.7/dist-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 1.1224
MAE:  0.8744
------------
Fold 2
RMSE: 1.1239
MAE:  0.8755
------------
Fold 3
RMSE: 1.1227
MAE:  0.8742
------------
Fold 4
RMSE: 1.1216
MAE:  0.8730
------------
Fold 5
RMSE: 1.1235
MAE:  0.8749
------------
------------
Mean RMSE: 1.1228
Mean MAE : 0.8744
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'mae': [0.8744362843418302,
                             0.8754638139810607,
                             0.8742434348428261,
                             0.8730157619832355,
                             0.8749001629049672],
                            'rmse': [1.1224458472371888,
                             1.12390445957997,
                             1.1226840933399709,
                             1.121590222798207,
                             1.1234891540230896]})

# Bias and Learning Rate

In [ ]:
#bias and learning rate
param_grid = {'biased': [True, False], 'lr_all': [0.001, 0.005, 0.05]}
#get accuracy/loss of hyperparameters by crossvalidation using all possible combinations
#cv-> crossvalidation splitting
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3, refit=True)

In [ ]:
#fit the best suited hyperparameters combination
gs.fit(data)

In [ ]:
#get fitted data
results = gs.cv_results

In [ ]:
#convert into dataframe to make it pretty
results_df = pd.DataFrame.from_dict(gs.cv_results)
results_df.to_csv('results.csv')

In [ ]:
#REMEMBER: LOWER IS BETTER!
results_df

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_biased,param_lr_all
0,1.326912,1.326454,1.326641,1.326669,0.000188,4,1.092232,1.092053,1.091901,1.092062,0.000135,4,398.091424,37.300258,38.866286,5.682648,"{'biased': True, 'lr_all': 0.001}",True,0.001
1,1.172019,1.172411,1.172351,1.172260,0.000172,2,0.922317,0.922771,0.922717,0.922601,0.000203,3,431.244209,6.205414,39.894010,1.245309,"{'biased': True, 'lr_all': 0.005}",True,0.005
2,1.070475,1.072364,1.070430,1.071090,0.000901,1,0.687729,0.689196,0.687951,0.688292,0.000645,1,430.122352,4.349267,42.572486,0.824236,"{'biased': True, 'lr_all': 0.05}",True,0.050
3,3.067546,3.065077,3.067049,3.066557,0.001066,6,2.705091,2.702365,2.704709,2.704055,0.001205,6,427.499279,4.514319,35.706372,3.069608,"{'biased': False, 'lr_all': 0.001}",False,0.001
4,2.049098,2.048767,2.033477,2.043781,0.007287,5,1.470876,1.471461,1.459443,1.467260,0.005533,5,423.939563,3.979564,36.649282,0.421633,"{'biased': False, 'lr_all': 0.005}",False,0.005
5,1.295606,1.291621,1.292479,1.293235,0.001713,3,0.838509,0.836631,0.836594,0.837245,0.000894,2,339.227206,86.084993,26.608742,9.423176,"{'biased': False, 'lr_all': 0.05}",False,0.050


# Tuning n factors

In [ ]:
#n_factors = n folds(deciding the best number of folds)
#finding the best combinations again but this time with n_factors
param_grid = {'n_factors': [1, 5, 10], 'biased': [True], 'lr_all': [0.005],
              'n_epochs': [20]}

gs2 = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

In [ ]:
gs2.fit(data)

In [ ]:
results = gs2.cv_results

In [ ]:
results_df = pd.DataFrame.from_dict(results)
results_df

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_factors,param_biased,param_lr_all,param_n_epochs
0,1.299486,1.298661,1.300259,1.299469,0.000652,3,1.053318,1.053491,1.054527,1.053779,0.000534,3,76.561859,1.281407,27.330688,0.763833,"{'n_factors': 1, 'biased': True, 'lr_all': 0.0...",1,True,0.005,20
1,1.287301,1.286533,1.287797,1.287210,0.000520,2,1.041877,1.042086,1.042851,1.042271,0.000418,2,88.908903,1.414167,28.405005,0.858258,"{'n_factors': 5, 'biased': True, 'lr_all': 0.0...",5,True,0.005,20
2,1.274665,1.274006,1.275742,1.274804,0.000716,1,1.029740,1.030041,1.031317,1.030366,0.000683,1,86.795732,21.132431,24.862788,4.517850,"{'n_factors': 10, 'biased': True, 'lr_all': 0....",10,True,0.005,20


# Tuning Epochs

In [ ]:
#n_epoch = epochs(deciding the best number of epoch)
#finding the best combinations again but this time with n_epoch
param_grid = {'n_epochs': [10, 20, 50], 'n_factors': [1], 'biased': [True], 'lr_all': [0.005]}
gs3 = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

In [ ]:
gs3.fit(data)

In [ ]:
results = gs3.cv_results
results_df = pd.DataFrame.from_dict(results)
results_df

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_epochs,param_n_factors,param_biased,param_lr_all
0,1.317589,1.317842,1.317499,1.317643,0.000145,3,1.077473,1.078517,1.078087,1.078026,0.000429,3,38.041055,0.943994,27.841902,0.534445,"{'n_epochs': 10, 'n_factors': 1, 'biased': Tru...",10,1,True,0.005
1,1.299465,1.299947,1.299685,1.299699,0.000197,2,1.053342,1.054449,1.054158,1.053983,0.000469,2,75.213372,2.515087,27.378492,0.937077,"{'n_epochs': 20, 'n_factors': 1, 'biased': Tru...",20,1,True,0.005
2,1.273742,1.274352,1.273943,1.274012,0.000254,1,0.998425,0.999843,0.999517,0.999262,0.000607,1,186.299065,40.385525,30.308682,9.477992,"{'n_epochs': 50, 'n_factors': 1, 'biased': Tru...",50,1,True,0.005


# Tuning Regressor


In [ ]:
param_grid = {'n_epochs': [20], 'n_factors': [1], 'biased': [True],
              'lr_all': [0.005], 'reg_all': [0.05, 0.06, 0.07]}
gs4 = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

In [ ]:
gs4.fit(data)

In [ ]:
results = gs4.cv_results
results_df = pd.DataFrame.from_dict(results)
results_df

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_epochs,param_n_factors,param_biased,param_lr_all,param_reg_all
0,1.301553,1.301891,1.299462,1.300969,0.001074,1,1.057828,1.057791,1.056431,1.057350,0.000650,1,74.965753,2.179578,27.666857,1.135614,"{'n_epochs': 20, 'n_factors': 1, 'biased': Tru...",20,1,True,0.005,0.05
1,1.301798,1.302021,1.299604,1.301141,0.001091,2,1.058715,1.058584,1.057252,1.058184,0.000661,2,73.006940,0.217685,26.868512,0.739908,"{'n_epochs': 20, 'n_factors': 1, 'biased': Tru...",20,1,True,0.005,0.06
2,1.301958,1.302337,1.299813,1.301369,0.001111,3,1.059511,1.059532,1.058091,1.059045,0.000674,3,61.835288,16.904103,23.979925,4.169716,"{'n_epochs': 20, 'n_factors': 1, 'biased': Tru...",20,1,True,0.005,0.07


In [ ]:
results_df[['rank_test_rmse', 'rank_test_mae']]

,rank_test_rmse,rank_test_mae
0,1,1
1,2,2
2,3,3


# Single Value Decomposition

In [ ]:
svdpp = SVD()

In [ ]:
evaluate(svdpp, data, measures=['RMSE', 'MAE'])

In [ ]:
nmf = NMF()

In [ ]:
evaluate(nmf, data, measures=['RMSE', 'MAE'])

# best model

In [ ]:
final = SVD(n_epochs=20, n_factors=1, biased=True,
              lr_all=0.005, reg_all=0.06)

In [ ]:
data = Dataset.load_from_file(file_path, reader=reader)
trainset, testset = train_test_split(data, test_size=.2)

In [ ]:
import time
start = time.time()

In [ ]:
final.fit(trainset)
runtime = time.time() - start
print(runtime)

112.86427211761475


In [ ]:
predictions = final.test(testset)

In [ ]:
accuracy.rmse(predictions)

RMSE: 1.2908


1.2908157217865033

In [ ]:
accuracy.mae(predictions)

MAE:  1.0480


1.0479786062486418

In [ ]:
dump.dump('finalmodel', algo=final, predictions=predictions, verbose=1)

The dump has been saved as file finalmodel


In [ ]:
preds, model = dump.load('finalmodel')

# Prediction

In [ ]:
model.predict('30678', '51871')

Prediction(uid='30678', iid='51871', r_ui=None, est=3.2966408389412387, details={'was_impossible': False})

# Mapping restaurant information to item IDs

In [ ]:
def find(f, seq):
    for item in seq:
        if f(item):
            return item

In [ ]:
def get_info(iid):
    return find(lambda b: iid == b['id'], businesses)

In [ ]:
import gc
gc.collect()

209

In [ ]:
'''import json
with open('/content/drive/MyDrive/Project/SEM 5/Yelp Dataset/FilteredData/finalbusinessesindexed.json') as f:
    businesses = json.load(f)'''
businesses=[]
import json
with open('/content/drive/MyDrive/Project/SEM 5/Yelp Dataset/shortdata/FilteredData/finalbusinessesindexed.json') as f:
    businesses = json.load(f)
#businesses.append(temp)

In [ ]:
get_info(78)

{'address': '3150 Roswell Rd NW, Ste A3',
 'attributes': {'Alcohol': "u'full_bar'",
  'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': True, 'casual': False}",
  'BusinessAcceptsCreditCards': 'True',
  'BusinessParking': "{'garage': True, 'street': True, 'validated': False, 'lot': False, 'valet': True}",
  'Caters': 'True',
  'DogsAllowed': 'False',
  'GoodForMeal': "{'dessert': None, 'latenight': None, 'lunch': None, 'dinner': True, 'brunch': None, 'breakfast': None}",
  'HappyHour': 'True',
  'HasTV': 'False',
  'Music': "{'dj': False, 'background_music': False, 'no_music': False, 'jukebox': False, 'live': False, 'video': False, 'karaoke': False}",
  'NoiseLevel': "u'loud'",
  'OutdoorSeating': 'None',
  'RestaurantsDelivery': 'True',
  'RestaurantsGoodForGroups': 'True',
  'RestaurantsReservations': 'True',
  'RestaurantsTableService': 'True',
  'RestaurantsTakeOut': 'True',
  'WiFi'

In [ ]:
def get_n_preds(uid, n):
    ratings = []
    for i in range(1, 73101):
        pred = model.predict(str(uid), str(i))
        ratings.append((int(pred.iid), pred.est))
    ratingsdesc = sorted(ratings, reverse=True, key=lambda x: x[1])[:n]
    namedratings = [(get_info(r[0])['name'], r[1]) for r in ratingsdesc]
    return namedratings

In [ ]:
get_n_preds(44, 10)

In [ ]:
import csv
with open('/content/drive/MyDrive/Project/SEM 5/Yelp Dataset/FilteredData/finaldata.csv', 'r') as f:
    reader = csv.reader(f)
    reviews = list(reader)

In [ ]:
def get_reviewed_restaurants(uid, desc=True):
    userreviews = list(filter(lambda r: r[0] == str(uid), reviews))
    ratings = [r[2] for r in userreviews]
    restaurants = list(map(lambda r: get_info(int(r[1])), userreviews))
    names = [r['name'] for r in restaurants]
    if desc==True:
        return sorted(list(zip(names, ratings)), reverse=True, key=lambda x: x[1])
    return sorted(list(zip(names, ratings)), key=lambda x: x[1])

In [ ]:
get_reviewed_restaurants(8145)